In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import pandas as pd
import glob

In [42]:
class PerformancePredictor(nn.Module):

    def __init__(self):
        super(PerformancePredictor, self).__init__()
        self.out = nn.Linear(9,2)

    def forward(self, x):
        x = self.out(x)
        return x

In [49]:
latent_space = {'VKITTI':[-958.49698523, 695.48164506, -1084.16241455, 970.46093565],
                'KITTI':[-1166.18798407, -1011.57989291, 566.2600466, 1178.45907698],
                'Cityscapes':[-712.90806697, 689.07183134, 866.59906593, 1263.15702937],
                'Synthia':[-251.38219667, 636.03292455, 415.35867623, 1196.09288925]}

data_dir = 'Results/*csv'

in_data = []
out_data = []

for data in glob.glob(data_dir):
    nn_input = []
    num_layers = data.split("_")[-1].split(".")[0]
    num_layers = int(num_layers)
    in_dataset = data.split("_")[1].split("-")[0]
    out_dataset = data.split("_")[1].split("-")[1]
    nn_input.extend(latent_space[in_dataset])
    nn_input.extend(latent_space[out_dataset])
    nn_input.append(num_layers)
    in_data.append(nn_input)
    
    csv_data = np.array(pd.read_csv(data,delimiter=','))
    arr = csv_data[-1]
    cycle = (arr[5]+arr[9])/2
    idt = (arr[6]+arr[10])/2
    out_data.append(np.array([cycle, idt]))

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print('Using:', device)

if use_cuda:
    torch.cuda.empty_cache()

epochs = 500
log_interval = 50
lr = 1e-4
performance_predictor = PerformancePredictor().to(device).train()

optimizer = optim.Adam(performance_predictor.parameters(), lr=lr, betas=(0.5, 0.999))

L1 = nn.L1Loss().to(device)
loss_nn = 0

in_data = torch.tensor(in_data).to(device)
out_data = torch.tensor(out_data).to(device)
dataset_size = len(in_data)

for epoch in range(epochs):
    for data in in_data:
        optimizer.zero_grad()
        pred_out = performance_predictor(in_data)
        loss = L1(pred_out, out_data)
        loss.backward()
        optimizer.step()
        loss_nn += loss.item()
    if (epoch+1)%log_interval == 0:
        print('Epoch: {}'.format(epoch+1))
        print('NN Loss: {}'.format(loss_nn/(dataset_size*log_interval)))
        loss_nn = 0

Using: cuda
Epoch: 50
NN Loss: 231.68875549316405
Epoch: 100
NN Loss: 59.20874016716367
Epoch: 150
NN Loss: 1.9653911322497186
Epoch: 200
NN Loss: 0.13746940249488468
Epoch: 250
NN Loss: 0.12001891029732568
Epoch: 300
NN Loss: 0.11417866997775578
Epoch: 350
NN Loss: 0.11297912879359154
Epoch: 400
NN Loss: 0.1112929102565561
Epoch: 450
NN Loss: 0.11163633592071988
Epoch: 500
NN Loss: 0.11030296183767772
